In [3]:
import pandas as pd
from google.cloud import storage
import io
import os

def process_laposte_insee_csv():
    """
    Traite le fichier CSV LaPoste INSEE depuis Google Cloud Storage
    - Supprime les colonnes 'Ligne_5' et 'Libellé_d_acheminement'
    - Supprime les doublons basés sur Code_commune_INSEE + Code_postal
    """

    # Configuration du bucket et du fichier
    bucket_name = "mspr-data-ia-raw"
    file_path = "LaPoste_INSEE_CP.csv"

    try:
        # On enregistre le path de l'objet (dans un bucket S3)
        bucket_path = 'gs://mspr-data-ia-raw/LaPoste_INSEE_CP.csv'

        # On initie un dataframe avec Pandas
        df = pd.read_csv(bucket_path, sep=';', encoding='latin1')

        # Nettoyage des noms de colonnes (suppression du # en début)
        df.columns = df.columns.str.replace('^#', '', regex=True)
        print(f"Données chargées: {len(df)} lignes, {len(df.columns)} colonnes")
        print("Colonnes présentes:", list(df.columns))

        # Filtrage: codes postaux format 77XXX / 91XXX / 95XXX / ...
        # Liste des codes départements de l’Île-de-France (SAUF PARIS)
        codes_idf = ['77', '78', '91', '92', '93', '94', '95']

        # Dans la colonne 'Code_commune_INSEE' on remplit les valeur N/A puis on converti en string
        df = df[
            (df['Code_commune_INSEE'].str.len() == 5) &
            (df['Code_commune_INSEE'].str[:2].isin(codes_idf))
        ]

        # Affichage de quelques lignes pour vérification
        print("\nAperçu des données (Seine-et-Marne uniquement):")
        display(df.head())

        # Suppression des colonnes spécifiées
        columns_to_drop = ['Ligne_5', 'Libellé_d_acheminement']
        existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]

        if existing_columns_to_drop:
            df = df.drop(columns=existing_columns_to_drop)
            print(f"\nColonnes supprimées: {existing_columns_to_drop}")
        else:
            print("\nAucune colonne à supprimer trouvée")

        # Vérification des doublons avant nettoyage
        initial_count = len(df)
        duplicates_mask = df.duplicated(subset=['Code_commune_INSEE', 'Code_postal'], keep=False)
        duplicates_count = duplicates_mask.sum()

        if duplicates_count > 0:
            print(f"\nDoublons détectés: {duplicates_count} lignes avec même Code_commune_INSEE + Code_postal")
            print("Exemples de doublons:")
            display(df[duplicates_mask].head(10))

        # Suppression des doublons (garde la première occurrence)
        df_cleaned = df.drop_duplicates(subset=['Code_commune_INSEE', 'Code_postal'], keep='first')

        removed_count = initial_count - len(df_cleaned)
        print(f"\nNettoyage terminé:")
        print(f"- Lignes initiales: {initial_count}")
        print(f"- Lignes supprimées (doublons): {removed_count}")
        print(f"- Lignes finales: {len(df_cleaned)}")

        # Affichage des statistiques finales
        print(f"\nDataFrame final:")
        print(f"- Colonnes: {list(df_cleaned.columns)}")
        print(f"- Dimensions: {df_cleaned.shape}")

        # Aperçu des données nettoyées
        print("\nAperçu des données nettoyées:")
        display(df_cleaned.head(10))

        # Vérification finale - pas de doublons restants
        final_duplicates = df_cleaned.duplicated(subset=['Code_commune_INSEE', 'Code_postal']).sum()
        print(f"\nVérification finale: {final_duplicates} doublons restants")

        return df_cleaned

    except Exception as e:
        print(f"Erreur lors du traitement: {str(e)}")
        return None

def save_cleaned_data(df, output_path=None):
    """
    Sauvegarde les données nettoyées
    """
    if df is None:
        print("Aucune donnée à sauvegarder")
        return

    if output_path is None:
        output_path = "LaPoste_INSEE_CP_cleaned.csv"

    df.to_csv(output_path, index=False)
    print(f"Données sauvegardées dans: {output_path}")

# Exemple d'utilisation
if __name__ == "__main__":
    # Traitement du fichier
    df_cleaned = process_laposte_insee_csv()

    # Sauvegarde optionnelle
    if df_cleaned is not None:
        save_cleaned_data(df_cleaned)

        # Statistiques additionnelles
        print(f"\nStatistiques des codes postaux:")
        print(f"- Codes postaux uniques: {df_cleaned['Code_postal'].nunique()}")
        print(f"- Communes uniques: {df_cleaned['Code_commune_INSEE'].nunique()}")

        # Vérification de la cohérence
        multi_cp_communes = df_cleaned.groupby('Code_commune_INSEE')['Code_postal'].nunique()
        communes_multi_cp = multi_cp_communes[multi_cp_communes > 1]

        if len(communes_multi_cp) > 0:
            print(f"\nCommunes avec plusieurs codes postaux: {len(communes_multi_cp)}")
            print("Exemples:")
            for insee, cp_count in communes_multi_cp.head().items():
                commune_data = df_cleaned[df_cleaned['Code_commune_INSEE'] == insee]
                print(f"  {insee} ({commune_data['Nom_de_la_commune'].iloc[0]}): {cp_count} codes postaux")

Données chargées: 39192 lignes, 5 colonnes
Colonnes présentes: ['Code_commune_INSEE', 'Nom_de_la_commune', 'Code_postal', 'Libellé_d_acheminement', 'Ligne_5']

Aperçu des données (Seine-et-Marne uniquement):


,Code_commune_INSEE,Nom_de_la_commune,Code_postal,Libellé_d_acheminement,Ligne_5
33394,77001,ACHERES LA FORET,77760,ACHERES LA FORET,NaN
33395,77002,AMILLIS,77120,AMILLIS,NaN
33396,77003,AMPONVILLE,77760,AMPONVILLE,NaN
33397,77004,ANDREZEL,77390,ANDREZEL,NaN
33398,77005,ANNET SUR MARNE,77410,ANNET SUR MARNE,NaN



Colonnes supprimées: ['Ligne_5', 'Libellé_d_acheminement']

Doublons détectés: 158 lignes avec même Code_commune_INSEE + Code_postal
Exemples de doublons:


,Code_commune_INSEE,Nom_de_la_commune,Code_postal
33403,77010,AUBEPIERRE OZOUER LE REPOS,77720
33404,77010,AUBEPIERRE OZOUER LE REPOS,77720
33413,77020,BANNOST VILLEGAGNON,77970
33414,77020,BANNOST VILLEGAGNON,77970
33420,77026,BEAUCHERY ST MARTIN,77560
33421,77026,BEAUCHERY ST MARTIN,77560
33425,77031,BERNAY VILBERT,77540
33426,77031,BERNAY VILBERT,77540
33454,77059,BUSSY ST MARTIN,77600
33455,77059,BUSSY ST MARTIN,77600



Nettoyage terminé:
- Lignes initiales: 1361
- Lignes supprimées (doublons): 85
- Lignes finales: 1276

DataFrame final:
- Colonnes: ['Code_commune_INSEE', 'Nom_de_la_commune', 'Code_postal']
- Dimensions: (1276, 3)

Aperçu des données nettoyées:


,Code_commune_INSEE,Nom_de_la_commune,Code_postal
33394,77001,ACHERES LA FORET,77760
33395,77002,AMILLIS,77120
33396,77003,AMPONVILLE,77760
33397,77004,ANDREZEL,77390
33398,77005,ANNET SUR MARNE,77410
33399,77006,ARBONNE LA FORET,77630
33400,77007,ARGENTIERES,77390
33401,77008,ARMENTIERES EN BRIE,77440
33402,77009,ARVILLE,77890
33403,77010,AUBEPIERRE OZOUER LE REPOS,77720



Vérification finale: 0 doublons restants
Données sauvegardées dans: LaPoste_INSEE_CP_cleaned.csv

Statistiques des codes postaux:
- Codes postaux uniques: 507
- Communes uniques: 1265

Communes avec plusieurs codes postaux: 10
Exemples:
  78551 (ST GERMAIN EN LAYE): 2 codes postaux
  91228 (EVRY COURCOURONNES): 2 codes postaux
  91479 (PARAY VIEILLE POSTE): 2 codes postaux
  92048 (MEUDON): 2 codes postaux
  93066 (ST DENIS): 3 codes postaux


In [4]:
from pandas_gbq import to_gbq

def export_to_bigquery_pandas_gbq(df, project_id, dataset_id, table_id):
    """
    Exporte un DataFrame vers BigQuery avec pandas-gbq

    Args:
        df: DataFrame à exporter
        project_id: ID du projet GCP
        dataset_id: ID du dataset BigQuery
        table_id: ID de la table BigQuery
    """

    # Configuration de la destination
    destination_table = f"{dataset_id}.{table_id}"

    try:
        # Export vers BigQuery
        to_gbq(
            df,
            destination_table=destination_table,
            project_id=project_id,
            if_exists='replace',  # Options: 'fail', 'replace', 'append'
            progress_bar=True,
            auth_local_webserver=False
        )

        print(f"✅ DataFrame exporté avec succès vers {project_id}.{destination_table}")
        print(f"📊 Nombre de lignes exportées : {len(df)}")
        print(f"📋 Nombre de colonnes : {len(df.columns)}")

    except Exception as e:
        print(f"❌ Erreur lors de l'export : {str(e)}")

df = df_cleaned
project_id = "mspr-data-ia"
dataset_id = "idf_silver_cleaned"
table_id = "dim_commune"
export_to_bigquery_pandas_gbq(df, project_id, dataset_id, table_id)

100%|██████████| 1/1 [00:00<00:00, 8738.13it/s]

✅ DataFrame exporté avec succès vers mspr-data-ia.idf_silver_cleaned.dim_commune
📊 Nombre de lignes exportées : 1276
📋 Nombre de colonnes : 3
